# LangChain 1.0 Basic(Real World) Agent in Databricks

Ref:
https://docs.langchain.com/oss/python/langchain/quickstart#build-a-real-world-agent

In [0]:
%pip install -U langchain>=1.0.0 langchain_openai>=1.0.0 databricks-langchain mlflow

%restart_python

## モデルとの接続

In [0]:
help(ChatOpenAI)

In [0]:
from langchain_openai import ChatOpenAI
from databricks_langchain import ChatDatabricks
import mlflow

mlflow.langchain.autolog()

# creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
# model = ChatOpenAI(
#     model="databricks-llama-4-maverick",
#     temperature=0.5,
#     timeout=10,
#     # max_tokens=1000,
#     # use_responses_api=False,
#     base_url=creds.host + "/serving-endpoints",
#     api_key=creds.token,
# )

model = ChatDatabricks(
    model="databricks-llama-4-maverick",
    temperature=0.5,
    timeout=10,
    max_tokens=1000,
)

## 単純なエージェントの作成と実行

In [0]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

In [0]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime


@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"


@dataclass
class Context:
    """Custom runtime context schema."""

    user_id: str


@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

In [0]:
from dataclasses import dataclass

# ここではdataclassを使用していますが、Pydanticモデルもサポートされています。
@dataclass
class ResponseFormat:
    """エージェントのレスポンススキーマ"""

    # ダジャレを含むレスポンス（必須）
    punny_response: str
    # 天気に関する興味深い情報（あれば）
    weather_conditions: str | None = None

In [0]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

In [0]:
from langchain.agents import create_agent
from pprint import pprint

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)

# `thread_id` は会話ごとの一意な識別子です。
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# 同じ `thread_id` を使って会話を続けることができます。
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )